In [2]:
from pathlib import Path
import pandas as pd
import dask.dataframe as dd

In [3]:
dataFolder = Path.cwd().joinpath('Data')

In [4]:
columnHeaders = ['bidID', 'Timestamp', 'logType', 'XYZID', 'useragent', 'ip', 'region', 'city', 'adexchange', 'domain', \
                 'url', 'anonURLID', 'adSlotID', 'width', 'height', 'visibility', \
                 'format', 'slotPrice', 'creativeId', 'bidprice', 'payPrice', 'keypageUrl', 'adverId', 'userTag']

In [5]:
impdataframes = [pd.read_csv(f, sep='\t', names=columnHeaders, compression='bz2') \
                 for f in dataFolder.glob("imp*.bz2")]

/home/sameer/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [6]:
impdf = pd.concat(impdataframes, axis=0)

In [8]:
clkdataframes = [pd.read_csv(f, sep='\t', names=columnHeaders, compression='bz2') \
                 for f in dataFolder.glob("clk*.bz2")]

In [9]:
clkdf = pd.concat(clkdataframes, axis=0)

In [10]:
clkdf[clkdf.bidID=='d503d08833987e6c792dc45ea1810800']

,bidID,Timestamp,logType,XYZID,useragent,ip,region,city,adexchange,domain,...,height,visibility,format,slotPrice,creativeId,bidprice,payPrice,keypageUrl,adverId,userTag


In [12]:
len(impdf.bidID[impdf.bidID.isin(clkdf.bidID)])

2716

In [13]:
len(impdf)

3158171

In [14]:
len(clkdf)

2716